# Introduction 

This notebook will generate a common employment data with following columns :
1. ID : Employe ID
2. DOB : Date of Birth
3. DOH : Date of Hire
4. Salary

In [1]:
import pandas as pd
import numpy as np
import datetime 

In [34]:
emp_counts = 50
data = pd.DataFrame(data = {"id" : np.random.randint(0, emp_counts*10, emp_counts), 
                            'dob' : [str(np.random.randint(1940, 1999)) + '-' + str(np.random.randint(1,12)) + '-' + str(np.random.randint(1,31)) for i in range (50)],
                            'doh' : [str(np.random.randint(2010, 2022)) + '-' + str(np.random.randint(1,12)) + '-' + str(np.random.randint(1,31)) for i in range (50)],
                            'salary' : np.round(np.random.randn(emp_counts)**2*1e3,2)})
data['dob'] = pd.to_datetime(data['dob'])
data['doh'] = pd.to_datetime(data['doh'])
data

,id,dob,doh,salary
0,317,1961-06-04,2020-06-22,7919.04
1,476,1988-03-25,2014-07-23,27.74
2,449,1987-11-01,2011-02-12,462.38
3,490,1943-04-28,2018-05-11,972.55
4,379,1960-09-23,2010-08-14,65.37
5,164,1986-07-02,2015-03-13,1524.39
6,101,1966-05-09,2012-06-30,315.80
7,442,1987-07-08,2020-05-09,235.28
8,36,1950-07-28,2016-02-07,128.11
9,36,1981-05-07,2020-10-24,1019.26


# Processing
We need the information of **Age** and **Year of Service (YOS)** of each of the employees.

In [50]:
val_date = pd.Timestamp('2022-12-31')
data['age'] = np.round((val_date- data.dob)/np.timedelta64(1, 'Y'),2)
data['yos'] = np.round((val_date- data.doh)/np.timedelta64(1, 'Y'),2)

In [51]:
data

,id,dob,doh,salary,age,yos
0,317,1961-06-04,2020-06-22,7919.04,61.58,2.52
1,476,1988-03-25,2014-07-23,27.74,34.77,8.44
2,449,1987-11-01,2011-02-12,462.38,35.17,11.88
3,490,1943-04-28,2018-05-11,972.55,79.68,4.64
4,379,1960-09-23,2010-08-14,65.37,62.27,12.38
5,164,1986-07-02,2015-03-13,1524.39,36.50,7.80
6,101,1966-05-09,2012-06-30,315.80,56.65,10.50
7,442,1987-07-08,2020-05-09,235.28,35.48,2.64
8,36,1950-07-28,2016-02-07,128.11,72.43,6.90
9,36,1981-05-07,2020-10-24,1019.26,41.65,2.18
